In [5]:
from osgeo import gdal
import os
from multiprocessing.pool import ThreadPool
import numpy as np

In [6]:
def generateFlipped(img):
    
    #open raster
    raster = gdal.Open(img, gdal.GA_ReadOnly)
    #get raster info
    rows = raster.RasterYSize
    cols = raster.RasterXSize
    bands = raster.RasterCount
    geo_transform = raster.GetGeoTransform()
    projection = raster.GetProjectionRef()
    
    #read raster as array
    rasterArray = raster.ReadAsArray()
    
    def flipArray(direction):
        axis = 1 if direction=='V' else 2 if direction=='H' else 3
 
        if axis in [1,2]: #vertical or horizontal
            if bands==1:
                ar_flipped = np.flip(rasterArray,axis-1)
            elif bands>1:
                ar_flipped = np.flip(rasterArray,axis)
        else: #vertical and horizontal
            if bands==1:
                ar_flipped = np.flip(np.flip(rasterArray,0),1)
            elif bands>1:
                ar_flipped = np.flip(np.flip(rasterArray,1),2)
                
        output_path = img.replace('.tif','_flipped_{}.tif'.format(direction))
        createGeotiff(output_path, ar_flipped, geo_transform, projection, bands)
    
    #create flipped imgs in parallel
    pool = ThreadPool()
    directions = ['V','H','VH']
    results = pool.map(flipArray, directions)
    
    raster = None
    

In [7]:
def createGeotiff(outRaster, data, geo_transform, projection, bands):
    # Create a GeoTIFF file with the given data
    driver = gdal.GetDriverByName('GTiff')
    if len(data.shape)==3:
        _ , rows, cols = data.shape
    elif len(data.shape)==2:
        rows, cols = data.shape
    rasterDS = driver.Create(outRaster, cols, rows, bands, gdal.GDT_Int32)
    rasterDS.SetGeoTransform(geo_transform)
    rasterDS.SetProjection(projection)
    if bands==1:
        band = rasterDS.GetRasterBand(1)
        band.WriteArray(data)
    else:
        for i in range(bands):
            band = rasterDS.GetRasterBand(i+1)
            band.WriteArray(data[i])
    rasterDS = None

In [9]:
img_folder = r"/media/daniel/7C42A09342A05424/Users/morae/Desktop/PhD_NovaIMS/Paper 3/Experimentos_DL/Imagem_Sentinel2/Inputs/train/data"
for img in os.listdir(img_folder):

    if img.endswith('.tif'):
        generateFlipped(os.path.join(img_folder,img))

In [26]:
arr = np.array([[1,2,3],[4,5,6],[7,8,9]])
arr


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [31]:
np.flip(arr,1)

array([[3, 2, 1],
       [6, 5, 4],
       [9, 8, 7]])